# BlogAutoWriter - 自動ブログ記事生成ツール

このJupyter Notebookは、OpenAI APIを使用して自動的にブログ記事を生成するツールです。

## 🎯 このツールでできること
- タイトルを入力するだけで、自動的にブログ記事を生成
- 複数の記事を一度に生成（並列処理で高速化）
- 記事の文体やスタイルをカスタマイズ可能
- 生成された記事をMarkdown形式で保存

## 📋 必要なもの
1. OpenAI APIキー
2. インターネット接続
3. Python 3.8以上

## 🚀 使い方
このノートブックのセルを上から順番に実行するだけです！
各セルの前に詳しい説明がありますので、安心して進めてください。

## ステップ1: 必要なライブラリをインストール

まず、記事生成に必要なPythonライブラリをインストールします。

**何をしているの？**
- `openai`: OpenAI APIと通信するためのライブラリ
- `pyyaml`: 設定ファイルを読み込むためのライブラリ
- `python-dotenv`: 環境変数を管理するためのライブラリ

**注意**: 初回実行時は少し時間がかかる場合があります。

In [ ]:
# 必要なライブラリをインストール
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 必要なパッケージのリスト
required_packages = [
    "openai",
    "pyyaml", 
    "python-dotenv"
]

print("📦 必要なライブラリをインストールしています...")
for package in required_packages:
    try:
        install_package(package)
        print(f"✅ {package} インストール完了")
    except Exception as e:
        print(f"❌ {package} のインストールに失敗: {e}")

print("\n🎉 ライブラリのインストールが完了しました！")

## ステップ2: 必要なモジュールをインポート

記事生成に必要なPythonモジュールを読み込みます。

**何をしているの？**
- 各種Pythonの標準ライブラリとインストールしたライブラリを読み込み
- エラーハンドリングも含めて、安全に実行できるようにしています

In [ ]:
# 必要なモジュールをインポート
import os
import json
import yaml
import time
import logging
import re
import unicodedata
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Any, Optional, Set
from concurrent.futures import ThreadPoolExecutor, as_completed

try:
    import openai
    print("✅ OpenAI ライブラリの読み込み完了")
except ImportError:
    print("❌ OpenAI ライブラリが見つかりません。上のセルを実行してインストールしてください。")
    raise

try:
    from dotenv import load_dotenv
    load_dotenv()  # .envファイルから環境変数を読み込み
    print("✅ 環境変数の読み込み完了")
except ImportError:
    print("⚠️ python-dotenvが見つかりません。環境変数は手動で設定してください。")

print("\n🎉 すべてのモジュールの読み込みが完了しました！")

## ステップ3: 設定管理クラスの定義

記事生成の設定を管理するクラスを定義します。

**何をしているの？**
- 記事の文体、対象読者、構成などの設定を管理
- 設定ファイル（JSON形式）の読み込みと作成
- デフォルト設定の提供

**カスタマイズ可能な項目**
- 文体・スタンス（例：丁寧で読みやすい文体）
- 対象読者（例：一般読者、専門家向け）
- 記事の長さ（セクション数、各セクションの文字数）
- OpenAI APIの設定（モデル、温度、最大トークン数）

In [ ]:
class ConfigManager:
    """設定管理クラス - 記事生成の設定を管理します"""
    
    # デフォルト設定
    DEFAULT_CONFIG = {
        "prompt_settings": {
            "style": "丁寧で読みやすい文体",
            "stance": "中立的",
            "target_audience": "一般読者",
            "article_length": {
                "sections": 3,
                "words_per_section": 300
            }
        },
        "openai": {
            "model": "gpt-4o-mini",  # 最新のモデル名に修正
            "temperature": 0.7,
            "max_tokens": 1000
        },
        "processing": {
            "max_threads": 10,
            "retry_attempts": 3,
            "retry_delay": 1.0
        }
    }
    
    def __init__(self, config_path: str = "config.json"):
        """設定管理クラスを初期化"""
        self.config_path = Path(config_path)
        self.config = self._load_config()
    
    def _load_config(self) -> Dict[str, Any]:
        """設定ファイルを読み込み、存在しない場合はデフォルト設定を作成"""
        if not self.config_path.exists():
            print(f"⚠️ 設定ファイル {self.config_path} が見つかりません。デフォルト設定を作成します。")
            self._create_default_config()
            return self.DEFAULT_CONFIG.copy()
        
        try:
            with open(self.config_path, 'r', encoding='utf-8') as f:
                if self.config_path.suffix.lower() in ['.yaml', '.yml']:
                    config = yaml.safe_load(f)
                else:
                    config = json.load(f)
            
            print(f"✅ 設定ファイル {self.config_path} を読み込みました")
            return self._merge_with_defaults(config)
        except Exception as e:
            print(f"❌ 設定ファイルの読み込みに失敗: {e}")
            raise ValueError(f"設定ファイルの読み込みに失敗しました: {e}")
    
    def _create_default_config(self):
        """デフォルト設定ファイルを作成"""
        self.config_path.parent.mkdir(parents=True, exist_ok=True)
        with open(self.config_path, 'w', encoding='utf-8') as f:
            json.dump(self.DEFAULT_CONFIG, f, ensure_ascii=False, indent=2)
        print(f"✅ デフォルト設定ファイル {self.config_path} を作成しました")
    
    def _merge_with_defaults(self, config: Dict[str, Any]) -> Dict[str, Any]:
        """読み込んだ設定をデフォルト設定とマージ"""
        merged = self.DEFAULT_CONFIG.copy()
        
        def deep_merge(default: dict, custom: dict):
            for key, value in custom.items():
                if key in default and isinstance(default[key], dict) and isinstance(value, dict):
                    deep_merge(default[key], value)
                else:
                    default[key] = value
        
        deep_merge(merged, config)
        return merged
    
    def get(self, key: str, default: Any = None) -> Any:
        """ドット記法で設定値を取得（例: 'openai.model'）"""
        keys = key.split('.')
        value = self.config
        
        for k in keys:
            if isinstance(value, dict) and k in value:
                value = value[k]
            else:
                return default
        
        return value
    
    def get_prompt_template(self) -> str:
        """設定に基づいてプロンプトテンプレートを生成"""
        settings = self.config['prompt_settings']
        return f"""以下の設定に従って、与えられたタイトルについて記事を執筆してください。

文体・スタンス: {settings['style']}、{settings['stance']}
対象読者: {settings['target_audience']}
記事構成: {settings['article_length']['sections']}つの見出しで構成し、各セクション約{settings['article_length']['words_per_section']}文字

タイトル: {{title}}

Markdown形式で出力し、以下の構造を守ってください：
# タイトル
## 見出し1
内容...
## 見出し2
内容...
## 見出し3
内容..."""

print("✅ ConfigManager クラスの定義が完了しました")

## ステップ4: ユーティリティ関数の定義

記事生成に必要な補助機能を定義します。

**何をしているの？**
- タイトルの検証（無効な文字のチェック、長さ制限）
- ファイル名の生成（日付_タイトル.md 形式）
- Markdownコンテンツの整形
- ディレクトリの作成と権限チェック

**安全性の確保**
- ファイルシステムに安全な文字のみを使用
- 長すぎるタイトルの自動調整
- 重複チェック機能

In [ ]:
def validate_title(title: str) -> Optional[str]:
    """タイトルを検証し、問題があればエラーメッセージを返す"""
    if not title or not title.strip():
        return "タイトルが空です"
    
    if len(title) > 200:
        return "タイトルが200文字を超えています"
    
    # ファイル名に使用できない文字をチェック
    invalid_chars = ['<', '>', ':', '"', '|', '?', '*', '\\', '/']
    for char in invalid_chars:
        if char in title:
            return f"無効な文字が含まれています: {char}"
    
    return None


def create_title_slug(title: str) -> str:
    """タイトルからファイル名用のスラッグを生成"""
    slug = title.strip()
    
    # Unicode正規化
    slug = unicodedata.normalize('NFKC', slug)
    
    # 無効な文字を削除
    slug = re.sub(r'[<>:"/\\|?*]', '', slug)
    
    # 空白文字をアンダースコアに変換
    slug = re.sub(r'[\s\u3000]+', '_', slug)
    
    # 先頭末尾のアンダースコアを削除
    slug = slug.strip('_')
    
    # 長すぎる場合は短縮
    if len(slug) > 50:
        slug = slug[:50].rstrip('_')
    
    # 空の場合はデフォルト名
    if not slug:
        slug = "untitled"
    
    return slug


def create_output_filename(title: str, date: Optional[datetime] = None) -> str:
    """出力ファイル名を生成（形式: YYYYMMDD_タイトルスラッグ.md）"""
    if date is None:
        date = datetime.now()
    
    date_str = date.strftime("%Y%m%d")
    slug = create_title_slug(title)
    
    return f"{date_str}_{slug}.md"


def sanitize_markdown_content(content: str) -> str:
    """Markdownコンテンツを整形・サニタイズ"""
    lines = content.split('\n')
    sanitized_lines = []
    
    for line in lines:
        line = line.rstrip()
        
        # 見出しの形式を正規化
        if line.startswith('#'):
            if not line.startswith('# ') and not line.startswith('## '):
                if line.startswith('#'):
                    line = '# ' + line[1:].strip()
                elif line.startswith('##'):
                    line = '## ' + line[2:].strip()
        
        sanitized_lines.append(line)
    
    content = '\n'.join(sanitized_lines)
    
    # 連続する空行を2つまでに制限
    content = re.sub(r'\n{3,}', '\n\n', content)
    
    return content.strip()


def ensure_directory_exists(path: Path) -> bool:
    """ディレクトリが存在することを確認、必要に応じて作成"""
    try:
        path.mkdir(parents=True, exist_ok=True)
        return True
    except Exception as e:
        print(f"❌ ディレクトリの作成に失敗: {e}")
        return False


def is_valid_output_directory(path: str) -> bool:
    """出力ディレクトリが有効で書き込み可能かチェック"""
    try:
        output_path = Path(path)
        
        if output_path.exists() and not output_path.is_dir():
            return False
        
        if not output_path.exists():
            output_path.mkdir(parents=True, exist_ok=True)
        
        # 書き込みテスト
        test_file = output_path / ".write_test"
        test_file.write_text("test")
        test_file.unlink()
        
        return True
    except Exception:
        return False

print("✅ ユーティリティ関数の定義が完了しました")

## ステップ5: OpenAI APIクライアントの定義

OpenAI APIと通信するためのクライアントクラスを定義します。

**何をしているの？**
- OpenAI APIへの接続と認証
- 記事生成のためのプロンプト送信
- エラーハンドリングと再試行機能
- レート制限への対応

**安全性の確保**
- APIキーの安全な管理
- 指数バックオフによる再試行
- 接続テスト機能

In [ ]:
class OpenAIClient:
    """OpenAI APIクライアント - 記事生成のためのAPI通信を管理"""
    
    def __init__(self, config: Dict[str, Any]):
        """OpenAI APIクライアントを初期化"""
        self.config = config
        
        # APIキーの取得
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            raise RuntimeError("❌ OPENAI_API_KEY環境変数が設定されていません")
        
        # OpenAI クライアントの初期化
        self.client = openai.OpenAI(api_key=api_key)
        
        # 設定値の取得
        self.model = config.get('openai', {}).get('model', 'gpt-4o-mini')
        self.temperature = config.get('openai', {}).get('temperature', 0.7)
        self.max_completion_tokens = config.get('openai', {}).get('max_tokens', 1000)
        self.max_retries = config.get('processing', {}).get('retry_attempts', 3)
        self.retry_delay = config.get('processing', {}).get('retry_delay', 1.0)
        
        print(f"✅ OpenAI APIクライアントを初期化しました（モデル: {self.model}）")
    
    def generate_article(self, prompt: str, title: str) -> Optional[str]:
        """OpenAI APIを使用して記事を生成"""
        formatted_prompt = prompt.format(title=title)
        
        for attempt in range(1, self.max_retries + 1):
            try:
                print(f"🔄 記事生成中... ({attempt}/{self.max_retries}): {title}")
                
                # API呼び出し
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {
                            "role": "system",
                            "content": "あなたは優秀なブログライターです。与えられたタイトルと設定に従って、読みやすく有益な記事を書いてください。"
                        },
                        {
                            "role": "user",
                            "content": formatted_prompt
                        }
                    ],
                    max_completion_tokens=self.max_completion_tokens,
                    temperature=self.temperature
                )
                
                # 応答の処理
                content = response.choices[0].message.content
                if content:
                    content = content.strip()
                    print(f"✅ 記事生成成功: {title} ({len(content)} 文字)")
                    return content
                else:
                    print(f"⚠️ 空の応答を受信: {title}")
                    return "記事の生成に失敗しました。"
                    
            except Exception as e:
                error_type = type(e).__name__
                print(f"❌ API呼び出し失敗 (試行 {attempt}/{self.max_retries}): {title} - {error_type}: {e}")
                
                if attempt < self.max_retries:
                    delay = self.retry_delay * (2 ** (attempt - 1))  # 指数バックオフ
                    print(f"⏳ {delay} 秒待機後に再試行します...")
                    time.sleep(delay)
                else:
                    print(f"❌ 最大試行回数に達しました。記事生成失敗: {title}")
                    return None
        
        return None
    
    def test_connection(self) -> bool:
        """OpenAI APIの接続テスト"""
        try:
            print("🔍 OpenAI API接続テストを実行中...")
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "user", "content": "Hello"}
                ],
                max_completion_tokens=10
            )
            
            if response.choices and response.choices[0].message.content:
                print("✅ OpenAI API接続テスト成功")
                return True
            else:
                print("❌ OpenAI API接続テスト失敗: 空の応答")
                return False
                
        except Exception as e:
            print(f"❌ OpenAI API接続テスト失敗: {e}")
            return False

print("✅ OpenAIClient クラスの定義が完了しました")

## ステップ6: 記事生成エンジンの定義

複数の記事を効率的に生成するメインエンジンを定義します。

**何をしているの？**
- 複数の記事を並列処理で同時生成
- 生成結果の管理とファイル保存
- 進捗の表示とエラーハンドリング
- 生成結果の統計情報の提供

**パフォーマンスの最適化**
- 最大10スレッドでの並列処理
- 効率的なタスク管理
- メモリ使用量の最適化

In [ ]:
class ArticleGenerator:
    """記事生成エンジン - 複数の記事を並列処理で効率的に生成"""
    
    def __init__(self, config_manager: ConfigManager):
        """記事生成エンジンを初期化"""
        self.config_manager = config_manager
        self.openai_client = OpenAIClient(config_manager.config)
        self.max_threads = config_manager.get('processing.max_threads', 10)
        print(f"✅ 記事生成エンジンを初期化しました（最大スレッド数: {self.max_threads}）")
    
    def generate_articles(self, titles: List[str], output_dir: Path) -> Dict[str, Dict[str, Any]]:
        """複数の記事を並列処理で生成"""
        results = {}
        
        # API接続テスト
        if not self.openai_client.test_connection():
            raise RuntimeError("❌ OpenAI API接続に失敗しました")
        
        # プロンプトテンプレートの取得
        prompt_template = self.config_manager.get_prompt_template()
        
        print(f"\n🚀 {len(titles)} 件の記事生成を開始します...")
        
        # 並列処理で記事生成
        with ThreadPoolExecutor(max_workers=self.max_threads) as executor:
            # 各タイトルに対してタスクを投入
            future_to_title = {
                executor.submit(
                    self._generate_single_article, 
                    title, 
                    prompt_template, 
                    output_dir
                ): title 
                for title in titles
            }
            
            # 完了したタスクから結果を取得
            completed_count = 0
            for future in as_completed(future_to_title):
                title = future_to_title[future]
                completed_count += 1
                
                try:
                    result = future.result()
                    results[title] = result
                    
                    if result['success']:
                        print(f"✅ [{completed_count}/{len(titles)}] 完了: {title}")
                    else:
                        print(f"❌ [{completed_count}/{len(titles)}] 失敗: {title} - {result['error']}")
                        
                except Exception as e:
                    error_msg = f"予期しないエラー: {e}"
                    results[title] = {
                        'success': False,
                        'error': error_msg,
                        'output_file': None
                    }
                    print(f"❌ [{completed_count}/{len(titles)}] 予期しないエラー: {title} - {error_msg}")
        
        return results
    
    def _generate_single_article(
        self, 
        title: str, 
        prompt_template: str, 
        output_dir: Path
    ) -> Dict[str, Any]:
        """単一の記事を生成"""
        try:
            # 記事内容を生成
            content = self.openai_client.generate_article(prompt_template, title)
            
            if not content:
                return {
                    'success': False,
                    'error': 'OpenAI APIから有効な応答を取得できませんでした',
                    'output_file': None
                }
            
            # コンテンツをサニタイズ
            sanitized_content = sanitize_markdown_content(content)
            
            # ファイル名を生成して保存
            filename = create_output_filename(title)
            output_file = output_dir / filename
            
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(sanitized_content)
            
            return {
                'success': True,
                'error': None,
                'output_file': str(output_file)
            }
            
        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'output_file': None
            }

print("✅ ArticleGenerator クラスの定義が完了しました")

## ステップ7: OpenAI APIキーの設定

記事生成にはOpenAI APIキーが必要です。

**APIキーの取得方法**
1. [OpenAI公式サイト](https://platform.openai.com/) にアクセス
2. アカウントを作成またはログイン
3. API Keys ページでAPIキーを生成
4. 下のセルでAPIキーを設定

**セキュリティ上の注意**
- APIキーは他人に教えないでください
- 使用後は適切に管理してください
- 不要になったら無効化することをお勧めします

**💡 ヒント**: APIキーは `sk-` で始まる長い文字列です

In [ ]:
# OpenAI APIキーの設定
import getpass

# 現在のAPIキーをチェック
current_api_key = os.getenv('OPENAI_API_KEY')

if current_api_key:
    # APIキーの最初の7文字と最後の4文字のみ表示（セキュリティのため）
    masked_key = current_api_key[:7] + '...' + current_api_key[-4:] if len(current_api_key) > 11 else '***'
    print(f"✅ 既存のAPIキーが設定されています: {masked_key}")
    
    use_existing = input("既存のAPIキーを使用しますか？ (y/n): ").strip().lower()
    if use_existing not in ['y', 'yes']:
        current_api_key = None

if not current_api_key:
    print("\n🔑 OpenAI APIキーを入力してください:")
    print("（入力したAPIキーは画面に表示されません）")
    
    while True:
        api_key = getpass.getpass("APIキー: ")
        
        if not api_key:
            print("❌ APIキーが入力されていません。再度入力してください。")
            continue
        
        if not api_key.startswith('sk-'):
            print("❌ 有効なOpenAI APIキーは 'sk-' で始まります。再度確認してください。")
            continue
        
        if len(api_key) < 20:
            print("❌ APIキーが短すぎます。正しいAPIキーを入力してください。")
            continue
        
        # 環境変数に設定
        os.environ['OPENAI_API_KEY'] = api_key
        print("✅ APIキーが正常に設定されました！")
        break
else:
    print("✅ 既存のAPIキーを使用します")

print("\n📝 注意: このAPIキーはこのセッション中のみ有効です。")
print("恒久的な設定には .env ファイルまたは環境変数を使用してください。")

## ステップ8: 設定の確認と初期化

記事生成の設定を確認し、必要なコンポーネントを初期化します。

**何をしているの？**
- 設定ファイルの読み込み
- 出力ディレクトリの作成
- 記事生成エンジンの初期化
- 現在の設定の表示

**カスタマイズ可能な設定**
- 出力ディレクトリ
- 記事の文体・構成
- OpenAI APIの設定
- 並列処理の設定

In [ ]:
# 設定とコンポーネントの初期化
print("⚙️ 設定を初期化しています...")

# 設定管理クラスの初期化
try:
    config_manager = ConfigManager("config.json")
    print("✅ 設定管理クラスの初期化完了")
except Exception as e:
    print(f"❌ 設定の初期化に失敗: {e}")
    raise

# 出力ディレクトリの設定
output_dir = Path("./output")
if ensure_directory_exists(output_dir):
    print(f"✅ 出力ディレクトリを確認しました: {output_dir}")
else:
    print(f"❌ 出力ディレクトリの作成に失敗: {output_dir}")
    raise RuntimeError("出力ディレクトリを作成できません")

# 記事生成エンジンの初期化
try:
    generator = ArticleGenerator(config_manager)
    print("✅ 記事生成エンジンの初期化完了")
except Exception as e:
    print(f"❌ 記事生成エンジンの初期化に失敗: {e}")
    raise

# 現在の設定を表示
print("\n" + "="*50)
print("📋 現在の設定")
print("="*50)
print(f"🤖 AIモデル: {config_manager.get('openai.model')}")
print(f"📝 文体: {config_manager.get('prompt_settings.style')}")
print(f"🎯 対象読者: {config_manager.get('prompt_settings.target_audience')}")
print(f"📊 記事構成: {config_manager.get('prompt_settings.article_length.sections')}セクション")
print(f"📁 出力先: {output_dir}")
print(f"⚡ 最大スレッド数: {config_manager.get('processing.max_threads')}")
print(f"🔄 再試行回数: {config_manager.get('processing.retry_attempts')}")
print("="*50)

print("\n🎉 すべての初期化が完了しました！")
print("次のセルでタイトルを入力して記事生成を開始できます。")

## ステップ9: 記事タイトルの入力

生成したい記事のタイトルを入力します。

**入力方法**
- 1行につき1つのタイトルを入力
- 入力を終了するには空行を入力または 'END' と入力
- 無効な文字や重複は自動的にチェックされます

**タイトルの注意点**
- 200文字以内で入力してください
- ファイル名に使用できない文字（< > : " | ? * \ /）は避けてください
- 分かりやすく具体的なタイトルにすると、より良い記事が生成されます

**💡 タイトル例**
- "Python初心者のための基本文法ガイド"
- "効果的なリモートワークのコツ"
- "健康的な食事の始め方"

In [ ]:
def collect_titles() -> List[str]:
    """ユーザーから記事タイトルを収集"""
    print("📝 記事タイトルを入力してください")
    print("   • 1行につき1つのタイトル")
    print("   • 空行または 'END' で入力終了")
    print("   • 最大200文字まで")
    print("-" * 40)
    
    titles = []
    seen_titles: Set[str] = set()
    empty_line_count = 0
    
    while True:
        try:
            prompt = f"タイトル {len(titles) + 1}: "
            line = input(prompt).strip()
            
            # 空行のチェック
            if not line:
                empty_line_count += 1
                if empty_line_count >= 2:
                    print("⚠️ 連続する空行が検出されました。入力を終了します。")
                    break
                print("💡 空行をもう一度入力すると終了します。タイトルを続けて入力してください。")
                continue
            
            # END コマンドのチェック
            if line.upper() == 'END':
                print("✅ 入力を終了します。")
                break
            
            empty_line_count = 0
            
            # タイトルの検証
            validation_error = validate_title(line)
            if validation_error:
                print(f"❌ {validation_error}")
                print("💡 もう一度正しいタイトルを入力してください。")
                continue
            
            # 重複チェック
            if line in seen_titles:
                print("⚠️ このタイトルは既に入力されています。")
                print("💡 別のタイトルを入力してください。")
                continue
            
            # タイトルを追加
            titles.append(line)
            seen_titles.add(line)
            print(f"   ✅ 追加されました: {line}")
            
        except KeyboardInterrupt:
            print("\n⚠️ 入力が中断されました。")
            break
        except EOFError:
            print("\n✅ 入力を終了します。")
            break
    
    print(f"\n📊 合計 {len(titles)} 件のタイトルが入力されました。")
    return titles

# タイトルの収集
titles = collect_titles()

if titles:
    print("\n" + "="*50)
    print("📋 入力されたタイトル一覧")
    print("="*50)
    for i, title in enumerate(titles, 1):
        print(f"{i:2d}. {title}")
    print("="*50)
else:
    print("⚠️ タイトルが入力されませんでした。")
    print("💡 上のセルを再実行してタイトルを入力してください。")

## ステップ10: 記事生成の実行

入力されたタイトルに基づいて記事を生成します。

**何が起こるの？**
- 各タイトルに対してOpenAI APIを呼び出し
- 複数の記事を並列処理で同時生成
- 生成された記事をMarkdown形式で保存
- 進捗状況をリアルタイムで表示

**生成にかかる時間**
- 1記事あたり約10-30秒（APIの応答速度に依存）
- 複数記事は並列処理で効率化
- ネットワークの状況により変動する場合があります

**⚠️ 注意事項**
- 生成中は画面を閉じないでください
- APIの利用料金が発生します
- 生成が失敗した場合は自動的に再試行されます

In [ ]:
if not titles:
    print("❌ タイトルが入力されていません。")
    print("💡 上のセルでタイトルを入力してから、このセルを実行してください。")
else:
    # 生成開始の確認
    print(f"🚀 {len(titles)} 件の記事生成を開始します。")
    print("\n⚠️ 重要な注意事項:")
    print("   • OpenAI APIの利用料金が発生します")
    print("   • 生成には時間がかかる場合があります")
    print("   • 生成中は画面を閉じないでください")
    
    # ユーザーの確認
    while True:
        try:
            response = input("\n記事生成を開始しますか？ (y/n): ").strip().lower()
            if response in ['y', 'yes', 'はい']:
                break
            elif response in ['n', 'no', 'いいえ']:
                print("❌ 記事生成をキャンセルしました。")
                titles = []  # タイトルをクリア
                break
            else:
                print("💡 'y' (はい) または 'n' (いいえ) で答えてください。")
        except (KeyboardInterrupt, EOFError):
            print("\n❌ 記事生成をキャンセルしました。")
            titles = []  # タイトルをクリア
            break
    
    # 記事生成の実行
    if titles:
        print("\n" + "="*60)
        print("🎬 記事生成を開始します！")
        print("="*60)
        
        start_time = time.time()
        
        try:
            # 記事生成の実行
            results = generator.generate_articles(titles, output_dir)
            
            end_time = time.time()
            total_time = end_time - start_time
            
            # 結果の集計
            successful = sum(1 for result in results.values() if result['success'])
            failed = len(results) - successful
            
            print("\n" + "="*60)
            print("📊 生成結果")
            print("="*60)
            
            # 成功した記事の表示
            if successful > 0:
                print("\n✅ 生成成功:")
                for title, result in results.items():
                    if result['success']:
                        print(f"   • {title}")
                        print(f"     → {result['output_file']}")
            
            # 失敗した記事の表示
            if failed > 0:
                print("\n❌ 生成失敗:")
                for title, result in results.items():
                    if not result['success']:
                        print(f"   • {title}")
                        print(f"     → {result['error']}")
            
            # 統計情報
            print(f"\n📈 統計情報:")
            print(f"   • 成功: {successful} 件")
            print(f"   • 失敗: {failed} 件")
            print(f"   • 所要時間: {total_time:.1f} 秒")
            print(f"   • 平均時間: {total_time/len(titles):.1f} 秒/記事")
            
            if successful > 0:
                print(f"\n🎉 {successful} 件の記事が正常に生成されました！")
                print(f"📁 生成されたファイルは {output_dir} フォルダに保存されています。")
            
            print("="*60)
            
        except Exception as e:
            print(f"\n❌ 記事生成中にエラーが発生しました: {e}")
            print("💡 設定を確認して、もう一度お試しください。")

## ステップ11: 生成された記事の確認

生成された記事ファイルを確認します。

**何をしているの？**
- 出力フォルダ内のファイル一覧を表示
- 各ファイルのサイズと作成日時を確認
- 記事の内容をプレビュー表示

**ファイル形式**
- ファイル名: `YYYYMMDD_タイトルスラッグ.md`
- 形式: Markdown (.md)
- 文字エンコーディング: UTF-8

**💡 活用方法**
- 生成されたMarkdownファイルはそのままブログに投稿可能
- 必要に応じて内容を編集・カスタマイズ
- 他のMarkdown対応ツールで開くことも可能

In [ ]:
# 生成された記事ファイルの確認
def display_generated_files(output_dir: Path):
    """生成された記事ファイルの一覧と情報を表示"""
    
    # Markdownファイルを検索
    md_files = list(output_dir.glob("*.md"))
    
    if not md_files:
        print("❌ 生成されたMarkdownファイルが見つかりません。")
        print(f"📁 出力フォルダ: {output_dir}")
        return
    
    # 作成日時でソート（新しい順）
    md_files.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    
    print("\n" + "="*60)
    print("📄 生成された記事ファイル一覧")
    print("="*60)
    
    for i, file_path in enumerate(md_files, 1):
        try:
            # ファイル情報を取得
            stat = file_path.stat()
            size_kb = stat.st_size / 1024
            created_time = datetime.fromtimestamp(stat.st_ctime)
            
            print(f"\n{i}. {file_path.name}")
            print(f"   📊 サイズ: {size_kb:.1f} KB")
            print(f"   🕐 作成日時: {created_time.strftime('%Y-%m-%d %H:%M:%S')}")
            
            # ファイル内容の最初の数行を表示
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    preview_lines = lines[:3]  # 最初の3行
                    
                print("   📝 プレビュー:")
                for line in preview_lines:
                    print(f"      {line.strip()}")
                    
                if len(lines) > 3:
                    print(f"      ... (続きは{len(lines)-3}行)")
                    
            except Exception as e:
                print(f"   ⚠️ プレビュー読み込みエラー: {e}")
                
        except Exception as e:
            print(f"   ❌ ファイル情報取得エラー: {e}")
    
    print(f"\n📁 全ファイル保存場所: {output_dir.absolute()}")
    print("="*60)

# ファイル一覧の表示
display_generated_files(output_dir)

# 特定のファイルを表示する機能
def show_article_content(output_dir: Path):
    """特定の記事の内容を全て表示"""
    md_files = list(output_dir.glob("*.md"))
    
    if not md_files:
        print("表示可能な記事がありません。")
        return
    
    print("\n📖 記事の内容を表示しますか？")
    print("表示したい記事の番号を入力してください（0で終了）:")
    
    # 作成日時でソート（新しい順）
    md_files.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    
    for i, file_path in enumerate(md_files, 1):
        print(f"  {i}. {file_path.name}")
    
    try:
        choice = input("\n番号を入力: ").strip()
        
        if choice == '0' or not choice:
            print("記事表示をスキップします。")
            return
        
        choice_num = int(choice)
        if 1 <= choice_num <= len(md_files):
            selected_file = md_files[choice_num - 1]
            
            print(f"\n" + "="*60)
            print(f"📄 {selected_file.name}")
            print("="*60)
            
            with open(selected_file, 'r', encoding='utf-8') as f:
                content = f.read()
                print(content)
            
            print("\n" + "="*60)
        else:
            print("❌ 無効な番号です。")
            
    except ValueError:
        print("❌ 数字を入力してください。")
    except (KeyboardInterrupt, EOFError):
        print("\n記事表示をキャンセルしました。")
    except Exception as e:
        print(f"❌ エラーが発生しました: {e}")

# 記事内容表示の実行
show_article_content(output_dir)

## ステップ12: 設定のカスタマイズ（オプション）

記事生成の設定を変更したい場合は、このセルを使用してください。

**カスタマイズ可能な項目**
- 文体・スタンス
- 対象読者
- 記事の長さ（セクション数、文字数）
- AIモデルの設定
- 処理設定（スレッド数など）

**💡 使用例**
- 専門的な記事: 「専門的で詳細な文体」「技術者向け」
- 初心者向け: 「分かりやすく親しみやすい文体」「初心者向け」
- ビジネス記事: 「簡潔でプロフェッショナルな文体」「ビジネスパーソン向け」

**注意**: 設定を変更した後は、記事生成エンジンを再初期化する必要があります。

In [ ]:
def customize_settings(config_manager: ConfigManager):
    """設定をカスタマイズ"""
    print("⚙️ 設定のカスタマイズ")
    print("現在の設定を表示し、変更したい項目を選択してください。")
    print("-" * 50)
    
    while True:
        # 現在の設定を表示
        print("\n📋 現在の設定:")
        print(f"1. 文体: {config_manager.get('prompt_settings.style')}")
        print(f"2. スタンス: {config_manager.get('prompt_settings.stance')}")
        print(f"3. 対象読者: {config_manager.get('prompt_settings.target_audience')}")
        print(f"4. セクション数: {config_manager.get('prompt_settings.article_length.sections')}")
        print(f"5. セクション文字数: {config_manager.get('prompt_settings.article_length.words_per_section')}")
        print(f"6. AIモデル: {config_manager.get('openai.model')}")
        print(f"7. 最大スレッド数: {config_manager.get('processing.max_threads')}")
        print("0. 設定完了")
        
        try:
            choice = input("\n変更する項目の番号を入力: ").strip()
            
            if choice == '0':
                break
            elif choice == '1':
                new_value = input("新しい文体を入力: ").strip()
                if new_value:
                    config_manager.config['prompt_settings']['style'] = new_value
                    print(f"✅ 文体を '{new_value}' に変更しました。")
            elif choice == '2':
                new_value = input("新しいスタンスを入力: ").strip()
                if new_value:
                    config_manager.config['prompt_settings']['stance'] = new_value
                    print(f"✅ スタンスを '{new_value}' に変更しました。")
            elif choice == '3':
                new_value = input("新しい対象読者を入力: ").strip()
                if new_value:
                    config_manager.config['prompt_settings']['target_audience'] = new_value
                    print(f"✅ 対象読者を '{new_value}' に変更しました。")
            elif choice == '4':
                try:
                    new_value = int(input("新しいセクション数を入力: ").strip())
                    if new_value > 0:
                        config_manager.config['prompt_settings']['article_length']['sections'] = new_value
                        print(f"✅ セクション数を {new_value} に変更しました。")
                    else:
                        print("❌ セクション数は1以上の数値で入力してください。")
                except ValueError:
                    print("❌ 数値を入力してください。")
            elif choice == '5':
                try:
                    new_value = int(input("新しいセクション文字数を入力: ").strip())
                    if new_value > 0:
                        config_manager.config['prompt_settings']['article_length']['words_per_section'] = new_value
                        print(f"✅ セクション文字数を {new_value} に変更しました。")
                    else:
                        print("❌ セクション文字数は1以上の数値で入力してください。")
                except ValueError:
                    print("❌ 数値を入力してください。")
            elif choice == '6':
                print("\n利用可能なモデル:")
                print("  • gpt-4o-mini (推奨・コスト効率)")
                print("  • gpt-4o (高品質)")
                print("  • gpt-3.5-turbo (高速)")
                new_value = input("新しいモデル名を入力: ").strip()
                if new_value:
                    config_manager.config['openai']['model'] = new_value
                    print(f"✅ モデルを '{new_value}' に変更しました。")
            elif choice == '7':
                try:
                    new_value = int(input("新しい最大スレッド数を入力 (1-20): ").strip())
                    if 1 <= new_value <= 20:
                        config_manager.config['processing']['max_threads'] = new_value
                        print(f"✅ 最大スレッド数を {new_value} に変更しました。")
                    else:
                        print("❌ スレッド数は1-20の範囲で入力してください。")
                except ValueError:
                    print("❌ 数値を入力してください。")
            else:
                print("❌ 無効な選択です。0-7の番号を入力してください。")
                
        except (KeyboardInterrupt, EOFError):
            print("\n設定カスタマイズを終了します。")
            break
    
    # 設定を保存
    try:
        config_manager.config_path.parent.mkdir(parents=True, exist_ok=True)
        with open(config_manager.config_path, 'w', encoding='utf-8') as f:
            json.dump(config_manager.config, f, ensure_ascii=False, indent=2)
        print(f"\n✅ 設定を {config_manager.config_path} に保存しました。")
    except Exception as e:
        print(f"❌ 設定の保存に失敗: {e}")

# 設定カスタマイズの実行
customize_choice = input("設定をカスタマイズしますか？ (y/n): ").strip().lower()

if customize_choice in ['y', 'yes', 'はい']:
    customize_settings(config_manager)
    
    # 設定変更後は記事生成エンジンを再初期化
    print("\n🔄 設定変更により記事生成エンジンを再初期化しています...")
    try:
        generator = ArticleGenerator(config_manager)
        print("✅ 記事生成エンジンの再初期化完了")
        print("💡 新しい設定で記事を生成するには、タイトル入力セルから再実行してください。")
    except Exception as e:
        print(f"❌ 記事生成エンジンの再初期化に失敗: {e}")
else:
    print("設定カスタマイズをスキップしました。")

## 🎉 完了！

お疲れさまでした！BlogAutoWriterを使った記事生成が完了しました。

### 📋 実行した処理のまとめ
1. ✅ 必要なライブラリのインストール
2. ✅ 設定管理システムの構築
3. ✅ OpenAI API連携の設定
4. ✅ 記事生成エンジンの初期化
5. ✅ タイトル入力と記事生成
6. ✅ 生成結果の確認

### 📁 生成されたファイル
- **記事ファイル**: `output/` フォルダ内の `.md` ファイル
- **設定ファイル**: `config.json`（カスタマイズした設定）

### 🔄 再実行方法
新しい記事を生成したい場合は、以下のセルから再実行してください：
- **タイトルだけ変更**: 「ステップ9: 記事タイトルの入力」から
- **設定も変更**: 「ステップ12: 設定のカスタマイズ」から

### 💡 活用のヒント
- 生成された記事は編集・カスタマイズが可能です
- 設定を調整することで、異なるスタイルの記事を生成できます
- 大量の記事を効率的に生成できます

### ❓ トラブルシューティング
- **API エラー**: OpenAI APIキーと残高を確認
- **生成が遅い**: ネットワーク接続とAPI制限を確認
- **設定エラー**: `config.json` ファイルの内容を確認

ご質問やお困りのことがありましたら、各セルのコメントや解説を参考にしてください。

**Happy Blogging! 📝✨**